# Data Exploring

---

In [181]:
import geopy
from geopy.geocoders import Nominatim
import geopandas as gpd
import shapefile
import rasterio
import rasterio.features
import rasterio.warp
from arcgis.gis import GIS

---

## PyShp

- NULL = 0
- POINT = 1
- POLYLINE = 3
- POLYGON = 5
- MULTIPOINT = 8
- POINTZ = 11
- POLYLINEZ = 13
- POLYGONZ = 15
- MULTIPOINTZ = 18
- POINTM = 21
- POLYLINEM = 23
- POLYGONM = 25
- MULTIPOINTM = 28
- MULTIPATCH = 31

In [7]:
sf = shapefile.Reader("./Data/DHMVIIDSMRAS1m_k01/DHMVII_vdc_k01/DHMVII_vdc_k01")

In [53]:
sf2 = shapefile.Reader("./Data/DHMVIIDSMRAS1m_k02/DHMVII_vdc_k02/DHMVII_vdc_k02")

In [89]:
sf3 = shapefile.Reader("./Data/DHMVIIDSMRAS1m_k03/DHMVII_vdc_k03/DHMVII_vdc_k03")

In [8]:
len(sf)

3

In [54]:
len(sf2)

7

In [90]:
len(sf3)

5

In [9]:
print(sf)

shapefile Reader
    3 shapes (type 'POLYGONZ')
    3 records (4 fields)


In [55]:
print(sf2)

shapefile Reader
    7 shapes (type 'POLYGON')
    7 records (4 fields)


In [91]:
print(sf3)

shapefile Reader
    5 shapes (type 'POLYGONZ')
    5 records (4 fields)


In [10]:
# the lower left (x,y) coordinate and upper right corner 

sf.bbox

[144900.0, 237758.6700000018, 162100.0300195366, 247224.38999511302]

In [128]:
type(sf.bbox)

shapefile._Array

In [56]:
sf2.bbox

[162000.0001220703, 238000.0001220703, 194000.0001220703, 250000.0001220703]

In [92]:
sf3.bbox

[193999.9999999993, 238000.00000000477, 206000.00000000017, 248000.00000000015]

In [41]:
xmin = sf.bbox[0]
ymin = sf.bbox[1]
xmax = sf.bbox[2]
ymax = sf.bbox[3]

In [13]:
shapes = sf.shape()

In [129]:
type(shapes)

shapefile.Shape

In [57]:
shapes2 = sf2.shape()

In [94]:
shapes3 = sf3.shape()

In [16]:
# The shape's index position in the original shapefile 

shapes.oid

0

In [95]:
shapes3.shapeType

15

In [93]:
shapes2.shapeType

5

In [18]:
shapes.shapeType

15

In [19]:
# If the shape record has multiple parts this attribute contains the index of the first point of each part.
# If there is only one part then a list containing 0 is returned 

shapes.parts

[0, 3344]

In [59]:
shapes2.parts

[0, 1288, 1609]

In [96]:
shapes3.parts

[0]

In [50]:
# contains a list of tuples containing an (x,y) coordinate for each point in the shape 

max(shapes.points)

(162100.0300195366, 246282.12998290733)

In [63]:
min(shapes2.points)

(162000.0001220703, 245951.89288330078)

In [23]:
len(shapes.points)

6268

In [60]:
len(shapes2.points)

1886

In [97]:
len(shapes3.points)

353

In [30]:
fields = sf.fields

In [61]:
fields2 = sf2.fields

In [98]:
fields3 = sf3.fields

In [33]:
fields

[('DeletionFlag', 'C', 1, 0),
 ['Id', 'N', 6, 0],
 ['OpnDatum1', 'D', 8, 0],
 ['OpnDatum2', 'D', 8, 0]]

In [62]:
fields2

[('DeletionFlag', 'C', 1, 0),
 ['Date', 'C', 254, 0],
 ['OpnDatum1', 'D', 8, 0],
 ['OpnDatum2', 'D', 8, 0]]

In [99]:
fields3

[('DeletionFlag', 'C', 1, 0),
 ['Date', 'C', 254, 0],
 ['OpnDatum1', 'D', 8, 0],
 ['OpnDatum2', 'D', 8, 0]]

In [37]:
records = sf.records()

In [69]:
records2 = sf2.records()

In [100]:
records3 = sf3.records()

In [38]:
len(records)

3

In [51]:
records

[Record #0: [0, datetime.date(2013, 2, 20), None],
 Record #1: [0, datetime.date(2013, 4, 1), None],
 Record #2: [0, datetime.date(2013, 2, 20), datetime.date(2013, 4, 1)]]

In [70]:
records2

[Record #0: ['20140129', datetime.date(2014, 1, 29), None],
 Record #1: ['20140130', datetime.date(2014, 1, 29), datetime.date(2014, 1, 30)],
 Record #2: ['20140130', datetime.date(2014, 1, 30), None],
 Record #3: ['20140131', datetime.date(2014, 1, 30), datetime.date(2014, 1, 31)],
 Record #4: ['20140131', datetime.date(2014, 1, 31), None],
 Record #5: ['20140202', datetime.date(2014, 1, 31), datetime.date(2014, 2, 2)],
 Record #6: ['20140202', datetime.date(2014, 2, 2), None]]

In [102]:
records3

[Record #0: ['20141121', datetime.date(2014, 11, 21), None],
 Record #1: ['20141124', datetime.date(2014, 11, 24), None],
 Record #2: ['20141125', datetime.date(2014, 11, 25), None],
 Record #3: ['20141121', datetime.date(2014, 11, 21), datetime.date(2014, 11, 24)],
 Record #4: ['20141125', datetime.date(2014, 11, 24), datetime.date(2015, 11, 25)]]

In [64]:
shapeRecs = sf.shapeRecords()

In [65]:
shapeRecs2 = sf2.shapeRecords()

In [68]:
print(shapeRecs)

ShapeRecords: [<shapefile.ShapeRecord object at 0x7f88c4758b20>, <shapefile.ShapeRecord object at 0x7f88c962dee0>, <shapefile.ShapeRecord object at 0x7f88c962dd60>]


---

## Rasterio

In [111]:
dataset = rasterio.open("./Data/DHMVIIDSMRAS1m_k01/GeoTIFF/DHMVIIDSMRAS1m_k01.tif")

In [174]:
dataset2 = rasterio.open("./Data/DHMVIIDSMRAS1m_k02/GeoTIFF/DHMVIIDSMRAS1m_k02.tif")

In [110]:
#with rasterio.open(
#    "./Data/DHMVIIDSMRAS1m_k01/GeoTIFF/DHMVIIDSMRAS1m_k01.tif"
#) as dataset:
#    mask = dataset.dataset_mask()
#    for geom, val in rasterio.features.shapes(mask, transform=dataset.transform):
#        geom = rasterio.warp.transform_geom(dataset.crs, "EPSG:4326", geom, precision=6)
#        print(geom)

In [112]:
dataset.name

'./Data/DHMVIIDSMRAS1m_k01/GeoTIFF/DHMVIIDSMRAS1m_k01.tif'

In [175]:
dataset2.name

'./Data/DHMVIIDSMRAS1m_k02/GeoTIFF/DHMVIIDSMRAS1m_k02.tif'

In [113]:
# Dataset objects have bands and this one has a band count of 1

dataset.count

1

In [176]:
dataset2.count

1

In [114]:
# A dataset band is an array of values
# All band arrays of a dataset have the same number of rows and columns
# width = columns 

dataset.width

17000

In [177]:
dataset2.width

32000

In [115]:
# height = rows 

dataset.height

9000

In [178]:
dataset2.height

12000

In [116]:
# Apply a dictionary comprehension to the zip() product of a dataset’s indexes and dtypes attributes
# to get a mapping of band indexes to variable data types 

{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'float32'}

In [117]:
dataset.bounds

BoundingBox(left=145000.0, bottom=238000.0, right=162000.0, top=247000.0)

In [179]:
dataset2.bounds

BoundingBox(left=162000.0, bottom=238000.0, right=194000.0, top=250000.0)

In [118]:
meters_wide = dataset.bounds[2] - dataset.bounds[0]
meters_high = dataset.bounds[3] - dataset.bounds[1]

In [119]:
print(f"{meters_wide} meters wide")
print(f"{meters_high} meters high")

17000.0 meters wide
9000.0 meters high


In [120]:
dataset.transform # * (0, 0)

Affine(1.0, 0.0, 145000.0,
       0.0, -1.0, 247000.0)

In [121]:
# These coordinate values are relative to the origin of the dataset’s Coordinate Reference System (CRS)

dataset.crs

CRS.from_epsg(31370)

### **EPSG 31370** refers to *Belgian Lambert 72 -- Belgium*

#### Ref. URL: https://epsg.io/31370

In [122]:
dataset.indexes

(1,)

In [123]:
band1 = dataset.read(1)

In [125]:
type(band1)

numpy.ndarray

In [126]:
band1

array([[-9.999e+03, -9.999e+03, -9.999e+03, ...,  1.275e+01,  1.279e+01,
         1.280e+01],
       [-9.999e+03, -9.999e+03, -9.999e+03, ...,  1.280e+01,  1.281e+01,
         1.284e+01],
       [-9.999e+03, -9.999e+03, -9.999e+03, ...,  1.284e+01,  1.283e+01,
         1.288e+01],
       ...,
       [ 4.520e+00,  4.550e+00,  4.570e+00, ...,  1.680e+01,  1.693e+01,
         1.690e+01],
       [ 4.490e+00,  4.570e+00,  4.580e+00, ...,  1.691e+01,  1.691e+01,
         1.695e+01],
       [ 4.480e+00,  4.550e+00,  4.570e+00, ...,  1.687e+01,  1.688e+01,
         1.691e+01]], dtype=float32)

To get the value for the pixel 10 kilometers east and 5 kilometers south of the dataset’s upper left corner, do the following:

In [142]:
dataset.bounds.left

145000.0

In [143]:
x, y = (dataset.bounds.left + 10000, dataset.bounds.top - 5000)

In [144]:
row, col = dataset.index(x, y)

In [145]:
row, col

(5000, 10000)

In [146]:
band1[row, col]

25.95

In [147]:
# To get the spatial coordinates of a pixel, use the dataset’s xy() method. 
# Ex.: the coordinates of the center of the image can be computed like this:

dataset.xy(dataset.height // 2, dataset.width // 2)

(153500.5, 242499.5)

---

## GeoPy

In [168]:
geopy.geocoders.ArcGIS()

In [169]:
geolocator = Nominatim(user_agent="ArcGIS")
location = geolocator.geocode("175 5th Avenue NYC")
print(location.address)

Flatiron Building, 175, 5th Avenue, Manhattan Community Board 5, Manhattan, New York, 10010, United States


In [170]:
print((location.latitude, location.longitude))

(40.741059199999995, -73.98964162240998)


In [171]:
location = geolocator.geocode("Brussels")

In [172]:
print(location.address)

Ville de Bruxelles - Stad Brussel, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, België / Belgique / Belgien


In [173]:
print((location.latitude, location.longitude))

(50.8465573, 4.351697)


---

## ArcGIS

In [150]:
print("ArcGIS Online as anonymous user")
gis = GIS()
print("Logged in as anonymous user to " + gis.properties.portalName)

ArcGIS Online as anonymous user
Logged in as anonymous user to ArcGIS Online


---

## GeoPandas

In [182]:
location_crs = location.to_crs(worldBound_robin.crs)

AttributeError: 'Location' object has no attribute 'to_crs'